# Elastic Collisions on Frictionless Surface

We have a small mass `m` and a big mass `M`. At the beginning `M` moves with velocity `v` towards `m`. The question is how many collisions `N` do we observe when $M = k \cdot m$ with $k > 1$.
And then further how does $N$ behave for $k \rightarrow \infty$

![sketch](small.jpg)

I am too lazy too derive these equations here, but looking here:
http://www.sciencecalculators.org/mechanics/collisions/

we see how the final velocities v1 and v2 are for m1 and m2 given their initial velocities u1 and u2, so lets write that into a little function

In [1]:
import numpy as np

In [2]:
def final_velocities(m1, m2, u1, u2):
    '''calculate final velocities after full elastic collision
    
    m1 and m2 : masses 
    u1 and u2 : initial velocities
    
    from: www.sciencecalculators.org/mechanics/collisions/
    '''

    v1 = (
        (
            u1 * (m1 - m2) + 2 * m2 * u2
        ) / (
            m1 + m2
        )
    )
    
    v2 = (
        (
            u2 * (m2 - m1) + 2 * m1 * u1
        ) / (
            m1 + m2
        )
    )
    
    
    return v1, v2

Now .. how do we go about this? 

For a numerical simulation, we need some numbers. So let's just assume: 

 * m = 1
 * k = 10, so M = 10
 * V = 1 (the initial speed of M)
 * v = 0 (the initial speed of m)
 
We also say positive velocities go towards the wall, and negative velocities go away from the wall.

So we have these cases:

 * v > V > 0: both masses run towards the wall, but m is faster, then m will collide with the wall and v will flip its sign.
 * v < 0 and V > 0: M is still heading towards the wall, and will soon collide with m.
 * v < V < 0: both masses run away from the wall, but m is faster and will still reach M and give it a bit more momentum.
 * V < v < 0: both masses run away from the wall, but M is faster and thus m will never collide with M again. 
 * V < 0 and v > 0: M runs away from the wall but m will collide with the wall next.

So lets try to implement these cases somehow 


In [3]:
m = 1
k = 10
M = k * m
V = 10
v = 0

collisions = 0

while True:
    if v > V and V >= 0: 
        # flip sign of v
        collisions += 1
        v = -v
    elif v <= 0 and V >= 0:
        collisions += 1
        v, V = final_velocities(m1=m, m2=M, u1=v, u2=V)
    elif v < V and V <= 0:
        collisions += 1
        v, V = final_velocities(m1=m, m2=M, u1=v, u2=V)
    elif V < v and v <= 0: 
        # end case        
        break
    elif V <= 0 and v >= 0:
        collisions += 1
        v = -v
    else:
        raise Exception('Cannot be')
    
print("Number of collisions", collisions)

Number of collisions 10


At this point I realize, I could have written this long `if...elif...else` block a bit shorter, so let's do that and put it into a function

In [4]:
def number_of_collisions(M, m=1, V=1, v=0):
    '''calculate the number of collisions between m and M 
    with a fixed wall in positive v direction.
    '''
    N = 0
    while True:
        if v > V:
            if v > 0:
                # m collides with wall
                v = -v
            else:
                # end case
                break
        else:
            # M collides with m
            v, V = final_velocities(m1=m, m2=M, u1=v, u2=V)
        
        # some kind of collision happened so we count up
        N += 1
    return N

After playing around with this, I realize that I remembered the original question wrongly ... we do not get 31 collisions when `k=10` .. but when `k=100` .. there is a factor 2 in the exponent I missed.

In [5]:
for half_exp in range(7):
    M = 10**(2*half_exp)
    N = number_of_collisions(M)
    # since m=1, k = M/m == M in this case...
    print(f'k:{M:.1e} -> N:{N}')

k:1.0e+00 -> N:3
k:1.0e+02 -> N:31
k:1.0e+04 -> N:314
k:1.0e+06 -> N:3141
k:1.0e+08 -> N:31415
k:1.0e+10 -> N:314159
k:1.0e+12 -> N:3141592


Oookay so we solved the question using a little simulation, BUT ... usually 3blue1brown expects us to find this answer using analytical methods ... so "real math" .. not the stuff I was doing here .. and honestly I have no idea how to solve this ... Here is the video
https://www.youtube.com/watch?v=HEfHFsfGXjs